<a href="https://colab.research.google.com/github/Kobra-A/a/blob/main/groupby_qid_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl https://msmarco.blob.core.windows.net/msmarcoranking/top1000.dev.tar.gz | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  655M  100  655M    0     0  1695k      0  0:06:36  0:06:36 --:--:-- 1658k


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

Sentence-Bert for Semantic Search

In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('msmarco-distilbert-base-v2')
from nltk.tokenize import sent_tokenize

In [7]:
!ls -al top1000.dev

-rwxrwx--- 1 8045835 1049089 2609106237 Jul 17  2019 top1000.dev


In [8]:
df = pd.read_csv('top1000.dev', delimiter='\t', names=('qid', 'pid', 'query', 'passage'))

In [9]:
gb = df.groupby('query')

1. Finding the most relevant sentence in each paragraph for each query

In [10]:
Query_embedding=[]
for name, group in tqdm(gb):
  Query_embedding.append(model.encode(name))

100%|██████████| 6980/6980 [00:55<00:00, 125.99it/s]


In [ ]:
passage_embedding=[]
Passage_Embedding=[]
count=100
for name, group in tqdm(gb):
  passage_embedding=[]
  print(len(group))
  if count>0:
    count=count-1
    for i in tqdm(range(0,len(group)-1)):
      passage_embedding.append(model.encode(sent_tokenize(group["passage"].iloc[i])))
    Passage_Embedding.append(passage_embedding)

In [45]:
print(len(Passage_Embedding))

100


In [ ]:
Max=[]
Index=[]
for i in tqdm(range(0,100)):
  max=[]
  index=[]
  for j in tqdm(range(0,len(Passage_Embedding[i]))):
    array=util.pytorch_cos_sim(Query_embedding[i], Passage_Embedding[i][j]).numpy()[0] #Fiding the cosine similarity of each query with each sentences in passage i 
    max.append(np.amax(array)) # Maximum Similarity in paragraph i
    index.append(np.argmax(array)) # index of the most similar sentence in the paragraph i
    #sentence.append(sent_tokenize(group["passage"].iloc[i])[np.argmax(array)]) # The most similar sentence in the paragraph i with query name
  Max.append(max)
  Index.append(index)
 # Sentence.append(sentence)

2. Rank paragraphs based on their cosine similarities

In [61]:
def Sort(array): #Sort the passages based on similarity and returns an array containing the index of passages in descending order
  temp = array.argsort() #index of ranked passages ascending similarity
  index_Des=temp[::-1] #index of ranked passages descending similarity---------------- #The one that I need
  Similarity_Des=array[temp[::-1]] #similarity of passages descending
  return index_Des

In [62]:
def rank(a):
  index=a
  Rank=[]
  for i in range(1,len(a)+1):
    Rank.append(0)
  for i in range(0,len(a)):
    Rank[a[i]]=i+1
  return Rank

In [63]:
Index_Desending=[] #index of sorted passages descending similarity
TOTAL_RANK=[] #rank of passages based on similarity
for i in range(0,len(Max)):
  array=np.array(Max[i]) # Similarity of first query with related passages
  index_Desending=Sort(array) #index of sorted passages descending similarity---------------- #The one that I need
  Index_Desending.append(index_Desending)
  RANK=rank(index_Desending)
  TOTAL_RANK.append(RANK)

In [65]:
print(TOTAL_RANK[0])

[53, 934, 343, 16, 90, 480, 683, 884, 932, 783, 771, 203, 92, 152, 263, 299, 286, 544, 554, 359, 61, 360, 156, 207, 528, 880, 550, 475, 655, 270, 517, 439, 617, 362, 728, 688, 593, 960, 609, 335, 275, 371, 539, 95, 946, 845, 485, 1, 162, 693, 968, 809, 361, 945, 28, 109, 898, 19, 434, 390, 196, 276, 944, 943, 542, 690, 127, 612, 640, 101, 42, 629, 20, 112, 941, 676, 518, 459, 43, 726, 600, 965, 738, 462, 857, 370, 260, 926, 35, 64, 412, 607, 519, 712, 826, 280, 453, 541, 810, 363, 314, 88, 21, 145, 549, 98, 159, 833, 143, 576, 199, 802, 707, 413, 449, 615, 469, 586, 878, 358, 921, 292, 818, 624, 460, 420, 639, 991, 369, 969, 63, 71, 200, 919, 301, 621, 515, 313, 454, 671, 896, 349, 598, 743, 433, 762, 735, 180, 936, 638, 663, 956, 815, 710, 753, 740, 808, 954, 995, 756, 583, 76, 331, 121, 975, 973, 146, 613, 493, 669, 308, 765, 424, 74, 183, 438, 854, 864, 703, 169, 430, 5, 450, 122, 102, 351, 788, 657, 667, 285, 822, 305, 774, 877, 933, 300, 421, 536, 579, 388, 741, 540, 401, 304, 581

Trash Code

In [ ]:
print(Max)
print(Max[0])
array=np.array(Max[0])# Similarity of first query with 16 related passages
print(array)

[[0.61036605, 0.22535923, 0.42747167, 0.7286943, 0.57226217, 0.38224673, 0.3298412, 0.2529214, 0.22593708, 0.2964775, 0.30064827, 0.49581128, 0.5683696, 0.51664656, 0.46575537], [0.25854397, 0.4761777, 0.15161176, 0.26833254, 0.4517699, 0.24665552, 0.08051856, 0.6155319, 0.38686323, 0.28173906, 0.45252255, 0.20701869, 0.270897, 0.47617766, 0.26362774], [0.72667575, 0.118471935, 0.16523194, 0.11741664, 0.22258708, 0.042314142, 0.032405637, 0.56356597, 0.09006925, 0.16555473, 0.21127138, 0.121809155, 0.18987468, 0.2246831, 0.08750952], [0.42326325, 0.21552673, 0.45964473, 0.74414533, 0.55340713, 0.6560862, 0.32511604, 0.4301777, 0.27784058, 0.24213766, 0.47393468, 0.21886736, 0.36782968, 0.30680376, 0.6162671], [0.14695317, 0.1775742, 0.19705506, 0.21594563, 0.14135745, 0.17641467, 0.16668266, 0.1225243, 0.45096165, 0.11937712, 0.20640044, 0.1989349, 0.24653798, 0.16409567, 0.35457623], [0.32588518, 0.31714845, 0.50079036, 0.4089254, 0.48534688, 0.41085374, 0.41859564, 0.34201115, 0.2818

In [ ]:
temp = array.argsort() #index of ranked passages ascending similarity
print(array)
print(temp[::-1]) #index of ranked passages descending similarity---------------- #The one that I need
print(array[temp[::-1]]) #similarity of passages descending

[0.23652679 0.3660336  0.3808589  0.47117764 0.2514261  0.2963557
 0.34114945 0.6191251  0.33654487 0.37067804 0.17735557 0.27203026
 0.5983861  0.46750742 0.45286268]
[ 7 12  3 13 14  2  9  1  6  8  5 11  4  0 10]
[0.6191251  0.5983861  0.47117764 0.46750742 0.45286268 0.3808589
 0.37067804 0.3660336  0.34114945 0.33654487 0.2963557  0.27203026
 0.2514261  0.23652679 0.17735557]


In [ ]:
Index1=temp[::-1]
Rank=rank(Index1)
print(Rank)

[14, 8, 6, 3, 13, 11, 9, 1, 10, 7, 15, 12, 2, 4, 5]


In [ ]:
Max=[] # Maximum Similarity of 100 query with 16 related passages
Index=[] # index of the most similar sentence
Sentence=[] # The most similar sentence in the paragraph
#count=100

for name, group in tqdm(gb):
#  if (count>0):
#    count=count-1
    max=[]
    index=[]
    sentence=[]
    query_embedding = model.encode(name)
    for i in tqdm(range(0,999)):
     # print(i)
     # print(group["passage"].iloc[i])
      passage_embedding = model.encode(sent_tokenize(group["passage"].iloc[i])) #tekenizing  passage i into sentences then embedding each sentence #it is a tensor (Matrix) 
      array=util.pytorch_cos_sim(query_embedding, passage_embedding).numpy()[0] #Fiding the cosine similarity of each query with each sentences in passage i 
      max.append(np.amax(array)) # Maximum Similarity in paragraph i
      index.append(np.argmax(array)) # index of the most similar sentence in the paragraph i
      sentence.append(sent_tokenize(group["passage"].iloc[i])[np.argmax(array)]) # The most similar sentence in the paragraph i with query name
    Max.append(max)
    Index.append(index)
    Sentence.append(sentence)